In [1]:
from ddpg_reacher import *
import warnings
warnings.filterwarnings("ignore")

In [2]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
env = gym.make("modified_gym_env:ReacherPyBulletEnv-v1", rand_init=False)
ddpg = DDPG(env, action_dim=2, state_dim=8, device=device, critic_lr=1e-3, actor_lr=1e-4, gamma=0.99, batch_size=100)

In [ ]:
ddpg.train(10000)

[Evaluate] Step_00150: mean_reward:-0.20122250152054738
[Evaluate] Step_00150: mean_reward:-0.20095191061196263
[Evaluate] Step_00150: mean_reward:-0.20042052536400057
[Evaluate] Step_00150: mean_reward:-0.1995742800756573
[Evaluate] Step_00150: mean_reward:-0.1984241657347666
[Evaluate] Step_00150: mean_reward:-0.1971068092401294
[Evaluate] Step_00150: mean_reward:-0.1958318444662601
[Evaluate] Step_00150: mean_reward:-0.19472988319122136
[Evaluate] Step_00150: mean_reward:-0.1938834152794396
[Evaluate] Step_00150: mean_reward:-0.19331609062994262
[Evaluate] Step_00150: mean_reward:-0.19303565336742284
[Evaluate] Step_00150: mean_reward:-0.19305289952608531
[Evaluate] Step_00150: mean_reward:-0.19337620525979823
[Evaluate] Step_00150: mean_reward:-0.19401305003512473
[Evaluate] Step_00150: mean_reward:-0.1950037601007737
[Evaluate] Step_00150: mean_reward:-0.19642111492167394
[Evaluate] Step_00150: mean_reward:-0.19833741541877328
[Evaluate] Step_00150: mean_reward:-0.2007281999899430

In [ ]:
# Train the policy
value_losses, policy_losses, validation_reward, validation_steps = ddpg.train(2e5)

plotting(validation_reward, "Average Rewards for Evaluation",
                            "eval_return.{}.png".format(SEED),"Iterations", "Reward")
plotting(validation_steps, "Steps to Completion",
                            "eval_steps_{}.png".format(SEED),"Iterations", "Steps to Completion")

torch.save(ddpg.actor.model,"./Actor.pth")
torch.save(ddpg.critic.model,"./Critic.pth")

np.save("validation_reward_{}.npy".format(SEED), validation_reward)
np.save("validation_steps_{}.npy".format(SEED), validation_steps)

# Evaluate the final policy
state, step, done = env.reset(), 0, False
env.render()
while not done:
    state = torch.from_numpy(state).float().unsqueeze(0).to(device)
    action = ddpg.actor(state).detach().squeeze().numpy()
    next_state, reward, done, _ = env.step(action)
    env.render()
    time.sleep(0.1)
    state = next_state
    step+=1
    print("Steps: {}, Action: {}, Reward: {}".format(step, action, reward))